# Scratch notebook for checking July initialization feedbacks

By: Ty Janoski
Updated: 10.13.21

## Set-up

In [2]:
# import statements

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from dask.diagnostics import ProgressBar

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [7]:
# read in cell area file for taking spatial averages
areacella = xr.open_dataarray('/dx01/janoski/cesm/output/gridarea.nc')

In [29]:
# create function for taking spatial averages, while weighting for latitude
def spav(ds_in, areacella=areacella, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of with coords lat and lon
    areacella -- Dataset or DataArray containing the grid cell area with coords lat and lon
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    ds_in = ds_in.sel(lat=slice(lat_bound_s,lat_bound_n))
    areacella = areacella.sel(lat=slice(lat_bound_s,lat_bound_n))
    out = (ds_in*(areacella/areacella.sum(dim=['lat','lon']))).sum(dim=['lat','lon'])
    return(out)

In [8]:
def preprocess(ds):
    dsnew = ds.copy()
    dsnew['time'] = np.arange(1,731,1)
    return dsnew

alb = xr.open_mfdataset('/dx02/janoski/cesm/vert_int_feedbacks/'+
                        'b40.1850.cam5-lens.07.*.h1_alb_rad_perturbs.nc',
                       preprocess=preprocess,combine='nested',concat_dim='ens')

In [23]:
arc = spatial_mean(alb,lat_bound_s=70)
with ProgressBar():
    arc = arc.compute()

[########################################] | 100% Completed |  3min  4.8s


In [30]:
arc2 = spav(alb,lat_bound_s=70)
with ProgressBar():
    arc2 = arc2.compute()

[########################################] | 100% Completed |  1min  7.5s


In [31]:
arc.FSNT.isel(ens=0).plot()
arc2.FSNT.isel(ens=0).plot(linestyle='dashdot')

<Figure size 432x288 with 1 Axes>